In [1]:
import numpy as np
import random
import copy
import torch
#import ElderNB
#user = ElderNB.Elder()

class StateTracker():
    """Tracks the state of the episode/conversation and prepares the state representation for the agent."""

    def __init__(self):
        """
        The constructor of StateTracker.
        """
        self.Objects = {'cup':['cup0','cup1','cup2'], 'ball':['ball0','ball1','ball2','ball3']}
        self.Locations = ['c1', 'd1', 'd2']
        self.error_p = 0.5
        self.reset()

    def reset(self):
        """Resets current_informs, history and round_num."""
        self.loc_obj = {}
        self.loc_obj['c1'] = {'cup':['cup0'], 'ball':['ball3']}
        self.loc_obj['d1'] = {'cup':['cup1'], 'ball':['ball0', 'ball1']}
        self.loc_obj['d2'] = {'cup':['cup2'], 'ball':['ball2']}

        self.agent_ot = [None, 0] #0:not filled, 1:mathces elder's, 2:doesn't match
        self.agent_l = [None, 0]
        self.agent_o = [None, 0]
        
        self.ot_utt_flag = False
        self.l_utt_flag = False
        
        
        self.user_l = self.Locations[random.randint(0,len(self.Locations)-1)]        
        self.user_ot = list(self.Objects.keys())[random.randint(0,1)]
        self.user_o = self.loc_obj[self.user_l][self.user_ot][random.randint(0,len(self.loc_obj[self.user_l][self.user_ot])-1)]
        #self.user_o = self.Objects[self.user_ot][random.randint(0,len(self.Objects[self.user_ot])-1)]

        self.agent_history = []
        self.user_history= []


    def get_state_agent(self):
        done = False
        success = False
        agent_output = self.agent_history[-1][0] if len(self.agent_history) >= 1 else None
        
        #prev_agent_output = self.agent_history[-2][0] if len(self.agent_history) > 1 else None
        #ot = self.agent_ot
        #l = self.agent_l
        #o = self.agent_o

        ot = self.agent_history[-1][1] if len(self.agent_history) >= 1 else [None,0]
        l = self.agent_history[-1][2] if len(self.agent_history) >= 1 else [None,0]
        o = self.agent_history[-1][3] if len(self.agent_history) >= 1 else [None,0]
        
        forbidden = self.agent_history[-1][4] if len(self.agent_history) >= 1 else None
        
        #if agent_output:
        #    if ((self.user_history[-1]['action_out']==6 or self.user_history[-1]['da_out']==7) and 
        #        agent_output['action_out']==5):
        #        self.agent_l = [None, 0]
        #        l = [None, 0]
        
        #prev_ot = self.agent_history[-2][1] if len(self.agent_history) > 1 else None
        #prev_l = self.agent_history[-2][2] if len(self.agent_history) > 1 else None
        #prev_o = self.agent_history[-2][3] if len(self.agent_history) > 1 else None
        if o[0] == self.user_o:
            done = True
            success = True
            o[1] = 1

        #return [[agent_output, prev_agent_output], [[ot, l, o], [prev_ot, prev_l, prev_o]], done, success, forbidden]
        return [agent_output, [ot, l, o], done, success, forbidden]

    def get_state_user(self):
        user_output = self.user_history[-1] if len(self.user_history) >= 1 else None
        #prev_user_output = self.user_history[-2] if len(self.user_history) > 1 else None

        return user_output

    def update_state_agent(self, helper_output):
        forbidden = False
        for key in helper_output:
            helper_output[key] = torch.argmax(helper_output[key])
            
        if helper_output['action_out'] in [3]:
            if not self.agent_ot[0]:
                forbidden = True
            else:
                #if self.agent_ot[0] == self.user_ot:
                self.agent_ot[0] = self.user_ot
                self.agent_ot[1] = 1
                self.ot_utt_flag = True
                    
        if helper_output['action_out'] in [4]:
            if not self.agent_l[0]:
                forbidden = True
            else:
                #if self.agent_l[0] == self.user_l:
                self.agent_l[0] = self.user_l
                self.agent_l[1] = 1
                self.l_utt_flag = True
            

        
        if helper_output['action_out'] in [5]: 
            if self.agent_l[0] and self.agent_ot[0]:
            ##if (self.agent_l == self.user_l) and (self.agent_ot == self.user_ot):
                if self.loc_obj[self.agent_l[0]][self.agent_ot[0]]:
                    self.agent_o[0] = self.loc_obj[self.agent_l[0]][self.agent_ot[0]][random.randint(0,len(self.loc_obj[self.agent_l[0]][self.agent_ot[0]])-1)]
                    self.agent_o[1] = 2
                    
            if (not self.agent_l[0]) or (not self.agent_ot[0]): #if (not self.agent_l[0]) or (not self.agent_ot[0])
                #self.agent_o = 'forbidden'
                forbidden = True
                
        #agent_l = [None, None]
        #if self.agent_l[0]:
        #    agent_l[0] = self.agent_l[0] + '*'
        #    agent_l[1] = self.agent_l[1]
        #agent_o = [None, None]
        #if self.agent_o[0]:
        #    agent_o[0] = self.agent_o[0] + '*'
        #    agent_o[1] = self.agent_o[1]
        #agent_ot = [None, None]
        #if self.agent_ot[0]:
        #    agent_ot[0] = self.agent_ot[0] + '*'
        #    agent_ot[1] = self.agent_ot[1]

        self.agent_history.append([helper_output, self.agent_ot, self.agent_l, self.agent_o, forbidden])

    def update_state_user(self, elder_output):
        for key in elder_output:
            elder_output[key] = torch.argmax(elder_output[key])
            
            
        alpha = random.random()

        if elder_output['action_out'] in [1, 3]:
            
            if self.error_p > alpha:
                if not self.ot_utt_flag:
                    self.agent_ot[0] = list(self.Objects.keys())[random.randint(0,1)]
                    self.agent_ot[1] = 2
                    self.ot_utt_flag = True
                    
            else:
                self.agent_ot[0] = self.user_ot
                self.agent_ot[1] = 1
                self.ot_utt_flag = True

                
                
        if elder_output['action_out'] in [2, 3]:
            if self.error_p <= alpha:
                if not self.l_utt_flag:
                    self.agent_l[0] = self.Locations[random.randint(0,len(self.Locations)-1)]
                    self.agent_l[1] = 2
                    self.ot_utt_flag = True
            
            else:
                self.agent_l[0] = self.user_l
                self.agent_l[1] = 1
                self.ot_utt_flag = True  


        #if elder_output['action_out'] in [1, 3]:
        #    if not self.ot_utt_flag:
        #        self.agent_ot[0] = list(self.Objects.keys())[random.randint(0,1)]
        #        self.agent_ot[1] = 2
        #        self.ot_utt_flag = True
        #    
        #    #infuse error
        #    #if self.error_p > random.random():
        #    #    self.agent_ot[0] = list(self.Objects.keys())[random.randint(0,1)]
        #    #    self.agent_ot[1] = 2
        #    #else:
        #    #    self.agent_ot[0] = self.user_ot
        #    #    self.agent_ot[1] = 1
        #        
        #        
        #if elder_output['action_out'] in [2, 3]:
        #    if not self.l_utt_flag:
        #        self.agent_l[0] = self.Locations[random.randint(0,len(self.Locations)-1)]
        #        self.agent_l[1] = 2
        #        self.ot_utt_flag = True
        #        
        #    #if self.error_p > random.random():
        #    #    self.agent_l[0] = self.Locations[random.randint(0,len(self.Locations)-1)]
        #    #    self.agent_l[1] = 2
        #    #else:
        #    #    self.agent_l[0] = self.user_l
        #    #    self.agent_l[1] = 1
            

        self.user_history.append(elder_output)
